# WASP Summer School 2020 
# Bayesian Optimization Tools Lab: HyperMapper 

### Objectives of this hands-on: 
1. Learn about a simple function used in the optimization literature: the Branin function
2. Use HyperMapper to optimize the 1D and 2D Branin functions
3. Explore HyperMapper's hyperparameters
3. Replace the Branin function with a function f(x) of your choice and optimize it 

### Prerequisites: 
In case you didn't go through the prerequisites of the module do it now: [Install](https://github.com/luinardi/hypermapper/wiki/Install-HyperMapper) Anaconda 3, HyperMapper and GPy.

Did you remember to export env variables before starting the Jupyter Notebook? If not do the following and restart this notebook: 

### General Setting: 
You are looking for a parameter setting that minimizes some performance metric of a function f(x) (such as error of an ML model, runtime, or cost). 

To use HyperMapper for this purpose you need to tell HyperMapper:
1. About your parameter space 
2. How to evaluate your algorithm's performance

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import json
import sys
import os
from matplotlib.lines import Line2D
%matplotlib inline

## #1 The Branin Function (https://www.sfu.ca/~ssurjano/branin.html) (5 minutes)

We look for minimizing the value of this function given the parameter $x_1 \in [-5, 10]$. $x_2$ is fixed for now: $x_2 = 2.275$ so that the Branin is 1D. The global minimum is at $x_1=\pi$.

This is the 1D Branin function that we want to minimize:

In [ ]:
def branin_function_1d(X):
    # The function must receive a dictionary, e.g., X = [{'x1': 2.3}]
    x1 = X['x1']
    
    # Branin function computation
    a = 1.0
    b = 5.1 / (4.0 * math.pi * math.pi)
    c = 5.0 / math.pi
    r = 6.0
    s = 10.0
    t = 1.0 / (8.0 * math.pi)
    x2 = 2.275
    value = a * (x2 - b * x1 * x1 + c * x1 - r) ** 2 + s * (1 - t) * math.cos(x1) + s

    # The function must return the objective value (a number)
    return value

Plot the 1D Branin and visualize the minimum:

In [ ]:
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 18
point_size = matplotlib.rcParams['lines.markersize']**2.8
point_size_optimum = matplotlib.rcParams['lines.markersize']**2

optimum = math.pi
value_at_optimum=branin_function_1d({'x1': optimum})

# Sample 1000 (x,y) pairs from the function to plot its curve
branin_line_xs = np.linspace(-5, 10, 1000)
branin_line_ys = []
for x in branin_line_xs:
    y = branin_function_1d({'x1': x})
    branin_line_ys.append(y)
plt.plot(branin_line_xs, branin_line_ys, label="1D Branin Function")

# Mark the known optimum on the curve
plt.scatter(optimum, value_at_optimum, s=point_size_optimum, marker='o', color="black", label="Minimum")

plt.legend()
plt.xlabel("x1")
plt.ylabel("value")
plt.show()
print("The 1d Branin function has one global optimum at x1 = \u03C0", flush=True)
print("(x, y) at minimum is: ("+str(optimum)+","+str(value_at_optimum)+")", flush=True)

## #2 Optimize 1D Branin Using HyperMapper (10 minutes)

Refer to the [HyperMapper](https://github.com/luinardi/hypermapper/wiki) wiki for question on the HyperMapper syntax.

You can find a list of topics on the menu on the right. [Here](https://github.com/luinardi/hypermapper/wiki/Json-Parameters-File) for example you can find a guide on the json parameters.

Specify the json file to run 1D Branin:

We create the json in python and then dump it on a file: 

In [ ]:
scenario = {}
scenario["application_name"] = "1d_branin"
scenario["optimization_objectives"] = ["value"]

number_of_RS = 2
scenario["design_of_experiment"] = {}
scenario["design_of_experiment"]["number_of_samples"] = number_of_RS

scenario["optimization_iterations"] = 15
    
scenario["models"] = {}
scenario["models"]["model"] = "gaussian_process"

scenario["input_parameters"] = {}
x1 = {}
x1["parameter_type"] = "real"
x1["values"] = [-5, 10]

scenario["input_parameters"]["x1"] = x1

with open("1d_branin.json", "w") as scenario_file:
    json.dump(scenario, scenario_file, indent=4)


To double check, print the json generated: 

In [ ]:
f = open("1d_branin.json", "r")
text = f.read()
print(text, flush=True)
f.close()

### Run HyperMapper
You are all set to run 1D Branin and HyperMapper together!

To optimize the branin function, call HyperMapper's optimize method with the json file and the branin function as parameters.

The execution ends with the message: "### End of the hypermapper script."

In [ ]:
import hypermapper
stdout = sys.stdout # Jupyter uses a special stdout and HyperMapper logging overwrites it. Save stdout to restore later
hypermapper.optimize(os. getcwd() + "/1d_branin.json", branin_function_1d)
sys.stdout = stdout

Output is in => "1d_branin_output_samples.csv". 

All the samples explored during optimization are in this file, check it out!

### Visualize

Visualize the points explored during optimization.

In [ ]:
cmap = plt.get_cmap('winter')
plt.plot(branin_line_xs, branin_line_ys, label="1D Branin Function")

# Load the points evaluated by HyperMapper during optimization
optimum = math.pi
sampled_points = pd.read_csv("1d_branin_output_samples.csv", usecols=['x1', 'value'])
x_points = sampled_points['x1'].values
y_points = sampled_points['value'].values

# Split between DoE and BO
doe_x = x_points[:number_of_RS]
doe_y = y_points[:number_of_RS]
bo_x = x_points[number_of_RS:]
bo_y = y_points[number_of_RS:]
bo_iterations = list(range(len(bo_x)))

plt.scatter(doe_x, doe_y, s=point_size, marker='x', color="red", label="Initial Random Sampling")
plt.scatter(optimum, value_at_optimum, s=point_size_optimum, marker='o', color="black", label="Minimum")
plt.scatter(bo_x, bo_y, s=point_size, marker='x', c=bo_iterations, cmap=cmap, label="Bayesian Otimization")

plt.legend()
plt.xlabel("x1")
plt.ylabel("value")
plt.show()

Blue points show points explored during optimization, with brighter points denoting points explored in later iterations. 

To double check, print the json generated: 

## 2D Branin

Optimize the full (2D) Branin function. $x_1 \in [-5, 10]$, $x_2 \in [0, 15]$.

In [ ]:
def branin_function(X):
    # Here the dictionary contains both input values
    x1 = X['x1']
    x2 = X['x2']
    
    a = 1.0
    b = 5.1 / (4.0 * math.pi * math.pi)
    c = 5.0 / math.pi
    r = 6.0
    s = 10.0
    t = 1.0 / (8.0 * math.pi)

    value = a * (x2 - b * x1 * x1 + c * x1 - r) ** 2 + s * (1 - t) * math.cos(x1) + s

    return value

### Setup HyperMapper to Run on 2d Branin

We create a json for the new configuration in python and dump it on a file.

In [ ]:
scenario = {}
scenario["application_name"] = "branin"
scenario["optimization_objectives"] = ["value"]

scenario["optimization_iterations"] = 20

scenario["models"] = {}
scenario["models"]["model"] = "random_forest"
scenario["models"]["number_of_trees"] = 20
    
scenario["input_parameters"] = {}
x1 = {}
x1["parameter_type"] = "real"
x1["values"] = [-5.0, 10.0]

x2 = {}
x2["parameter_type"] = "real"
x2["values"] = [0, 15.0]

scenario["input_parameters"]["x1"] = x1
scenario["input_parameters"]["x2"] = x2

with open("branin.json", "w") as scenario_file:
    json.dump(scenario, scenario_file, indent=4)

To double check, print the json generated: 

In [ ]:
f = open("branin.json", "r")
text = f.read()
print(text, flush=True)
f.close()

### Run HyperMapper

The execution ends with the message: "### End of the hypermapper script."

In [ ]:
hypermapper.optimize(os. getcwd() + "/branin.json", branin_function)
sys.stdout = stdout

### Visualize

Visualize the points explored during optimization.

In [ ]:
# Plot the value of the Branin function over a grid of the space
heatmap_samples = 100
x1_heatmap_values = np.linspace(-5, 10, 100)
x2_heatmap_values = np.linspace(0, 15, 100)
branin_values = np.zeros((heatmap_samples, heatmap_samples), dtype=float)
for i, x1 in enumerate(x1_heatmap_values):
    for j, x2 in enumerate(x2_heatmap_values):
        branin_values[j,i] = branin_function({'x1': x1, 'x2': x2})

heat_cmap = plt.get_cmap("gist_heat_r")
cf = plt.pcolormesh(x1_heatmap_values, x2_heatmap_values, branin_values, cmap=heat_cmap, alpha=0.5)
plt.colorbar(cf)

# Load the points evaluated by HyperMapper during optimization
sampled_points = pd.read_csv("branin_output_samples.csv", usecols=['x1', 'x2'])
x1_points = sampled_points['x1'].values
x2_points = sampled_points['x2'].values

# Split between DoE and BO
doe_x1 = x1_points[:10]
doe_x2 = x2_points[:10]
bo_x1 = x1_points[10:]
bo_x2 = x2_points[10:]
bo_iterations = list(range(len(bo_x1)))

optima_x1 = [-3.141, 3.141, 9.425]
optima_x2 = [12.275, 2.275, 2.475]
plt.scatter(optima_x1, optima_x2, s=point_size, marker='x', color="black", label="Minima")
plt.scatter(doe_x1, doe_x2, s=point_size**0.8, marker='x', color="red", label="Initial Random Sampling")
plt.scatter(bo_x1, bo_x2, s=point_size**0.8, marker='x', c=bo_iterations, cmap=cmap, label="Bayesian Otimization")

plt.xlim(min(x1_heatmap_values), max(x1_heatmap_values))
plt.ylim(min(x2_heatmap_values), max(x2_heatmap_values))
plt.xlabel('x1')
plt.ylabel('x2')

legend_elements = [
    Line2D([0], [0], marker='x', color='red', linestyle='None', label="Initial Random Sampling"),
    Line2D([0], [0], marker='x', color='blue', linestyle='None', label="Bayesian Optimization"),
    Line2D([0], [0], marker='x', color='black', linestyle='None', label='Minima'),
]
plt.legend(handles=legend_elements, bbox_to_anchor=(1.75, 1), fancybox=True, shadow=True, ncol=1)

plt.show()

Blue points show points explored during optimization, with brighter points denoting points explored in later iterations.

The Branin function has three optima at $(-\pi, 12.275)$, $(\pi, 2.275)$, and $(9.42478, 2.475)$.

## #3 Experiment With 2D Branin (5 minutes)

Experiment with the 2D Branin hacking the json file and running the code above. You can experiment with the following: 
1. Change the number of iniitialization steps (random samples):
"design_of_experiment": { "number_of_samples": 3 }

2. Change the number of optimization steps:
"optimization_iterations": 20,

3. Use a different acquisition function: 
"acquisition_function": "EI" or 
"acquisition_function": "UCB"  

What do you observe? 

## #4 Experiment With Your Favorite Black-box Function (10 minutes)

Perhaps you can chooose a classic function [here](https://www.sfu.ca/~ssurjano/index.html).

Or Maybe you can choose your favorite ML model and tune its hyperparameters. Try for example tuning the Random Forests model [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

It is your choice. 

# End of the Session
This is the end of the hands-on and of the whole session. If you have more questions, [ask](luigi.nardi@cs.lth.se) us.

And if you liked HyperMapper don't forget to star the [HyperMapper GitHub repo](https://github.com/luinardi/hypermapper/)!

Join the HyperMapper Slack channel hypermapper.slack.com! [Request access](luigi.nardi@cs.lth.se).